# Modulo II Practica 1 

In [1]:
import pickle
import association_metrics as am
import pandas as pd

import os
from kaggle.api.kaggle_api_extended import KaggleApi
import opendatasets as od
import session_info

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

import numpy as np
from scipy import stats
from scipy.stats import ks_2samp
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import RocCurveDisplay
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, cross_validate
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures

In [2]:
with open('COVID19MEXICO.pkl', 'rb') as file:
    df = pickle.load(file)

In [3]:
df

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2023-09-19,0ed066,2,12,16,2,16,16,41,1,...,2,2,97,1,2,7,99,México,97,97
1,2023-09-19,b4071e,2,12,9,1,9,9,3,1,...,2,2,97,1,2,7,99,México,97,97
2,2023-09-19,a5442c,2,12,9,1,9,9,4,1,...,2,2,97,1,2,7,99,México,97,97
3,2023-09-19,dacb2c,2,12,7,1,7,7,108,1,...,2,2,97,2,97,6,99,México,97,97
4,2023-09-19,6cef7b,2,12,7,1,7,7,108,1,...,2,2,97,2,97,6,99,México,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137994,2023-09-19,m00ee10,2,12,15,2,30,15,33,1,...,99,2,97,1,2,7,99,México,97,97
1137995,2023-09-19,m1cc581,2,3,15,1,20,15,999,1,...,99,2,97,1,2,7,99,México,97,97
1137996,2023-09-19,m0b0348,2,12,15,1,9,15,999,1,...,99,2,97,1,2,7,99,México,97,97
1137997,2023-09-19,m08f43c,2,3,15,2,15,15,999,1,...,99,2,97,1,2,7,99,México,97,97


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137999 entries, 0 to 1137998
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1137999 non-null  object
 1   ID_REGISTRO            1137999 non-null  object
 2   ORIGEN                 1137999 non-null  int64 
 3   SECTOR                 1137999 non-null  int64 
 4   ENTIDAD_UM             1137999 non-null  int64 
 5   SEXO                   1137999 non-null  int64 
 6   ENTIDAD_NAC            1137999 non-null  int64 
 7   ENTIDAD_RES            1137999 non-null  int64 
 8   MUNICIPIO_RES          1137999 non-null  int64 
 9   TIPO_PACIENTE          1137999 non-null  int64 
 10  FECHA_INGRESO          1137999 non-null  object
 11  FECHA_SINTOMAS         1137999 non-null  object
 12  FECHA_DEF              1137999 non-null  object
 13  INTUBADO               1137999 non-null  int64 
 14  NEUMONIA               1137999 non

In [5]:
df=df.drop(columns = ["FECHA_DEF","FECHA_INGRESO","INTUBADO","UCI"])

In [6]:
#Haciendo nuestra target dummy
df.loc[df['TIPO_PACIENTE']== 2,'TIPO_PACIENTE']  = 0
df.loc[df['TIPO_PACIENTE']== 1,'TIPO_PACIENTE']  = 1

In [7]:
df['TIPO_PACIENTE'].value_counts(1)

TIPO_PACIENTE
1    0.922579
0    0.077421
Name: proportion, dtype: float64

In [8]:
df['TIPO_PACIENTE'].unique()

array([1, 0])

In [9]:
#Como todas nuestras variables 
for i in df:
  print(i)
  values = df[i].value_counts(1)
  display(values)
  print("\n")

FECHA_ACTUALIZACION


FECHA_ACTUALIZACION
2023-09-19    1.0
Name: proportion, dtype: float64



ID_REGISTRO


ID_REGISTRO
0ed066     8.787354e-07
g3dc7a6    8.787354e-07
g2327ac    8.787354e-07
g2d6dab    8.787354e-07
g2d4e80    8.787354e-07
               ...     
g3e3dcc    8.787354e-07
g441930    8.787354e-07
g3808fe    8.787354e-07
g352c92    8.787354e-07
m1325ff    8.787354e-07
Name: proportion, Length: 1137999, dtype: float64



ORIGEN


...

MIGRANTE
99    0.996544
2     0.002131
1     0.001325
Name: proportion, dtype: float64



PAIS_NACIONALIDAD


PAIS_NACIONALIDAD
México                       9.957320e-01
Estados Unidos de América    8.435860e-04
Venezuela                    5.413010e-04
República de Honduras        3.145873e-04
China                        2.943764e-04
                                 ...     
Austria                      8.787354e-07
República Centro Africana    8.787354e-07
Níger                        8.787354e-07
Marruecos                    8.787354e-07
Bielorrusia                  8.787354e-07
Name: proportion, Length: 102, dtype: float64



PAIS_ORIGEN


PAIS_ORIGEN
97                                      9.986775e-01
Venezuela                               2.495609e-04
República de Honduras                   2.117752e-04
Haití                                   1.291741e-04
Nicaragua                               1.177505e-04
Guatemala                               7.205630e-05
Brasil                                  6.678389e-05
Estados Unidos de América               6.151148e-05
China                                   5.623907e-05
El Salvador                             5.360286e-05
Colombia                                5.184539e-05
Ecuador                                 5.096665e-05
Chile                                   4.569424e-05
Cuba                                    3.690689e-05
Otro                                    2.460459e-05
Afganistán                              1.054482e-05
Perú                                    8.787354e-06
Rusia                                   7.908619e-06
Bolivia                           

In [10]:
#Normalización de Sector menores a .01 se agruparán en otra categoria
df['SECTOR']=df["SECTOR"].map(lambda 
                                    x: 0 if x== 8 
                                    or x== 5 
                                    or x== 13 
                                    or x== 7
                                    or x== 11
                                    or x== 10
                                    or x== 2
                                    or x== 99
                                    or x== 1
                                    else x)

In [11]:
df['SECTOR'].value_counts(1)

SECTOR
4     0.536894
12    0.314426
9     0.076382
6     0.049733
3     0.014549
0     0.008015
Name: proportion, dtype: float64

In [12]:
df['ENTIDAD_NAC'] = df['ENTIDAD_NAC'].astype(str)
df['ENTIDAD_UM'] = df['ENTIDAD_UM'].astype(str)
df['ENTIDAD_RES'] = df['ENTIDAD_RES'].astype(str)

In [13]:
#ENTIDAD_NAC
diccionario_region = {
    "1": "Centro de Mexico",  # AGUASCALIENTES
    "2": "Norte de Mexico",    # BAJA CALIFORNIA
    "3": "Norte de Mexico",    # BAJA CALIFORNIA SUR
    "4": "Sur de Mexico",      # CAMPECHE
    "5": "Norte de Mexico",    # COAHUILA DE ZARAGOZA
    "6": "Occidente de Mexico", # COLIMA
    "7": "Sur de Mexico",      # CHIAPAS
    "8": "Norte de Mexico",    # CHIHUAHUA
    "9": "Centro de Mexico",   # CIUDAD DE MÉXICO
    "10": "Norte de Mexico",    # DURANGO
    "11": "Centro de Mexico",   # GUANAJUATO
    "12": "Sur de Mexico",      # GUERRERO
    "13": "Centro de Mexico",   # HIDALGO
    "14": "Occidente de Mexico", # JALISCO
    "15": "Centro de Mexico",   # MÉXICO
    "16": "Occidente de Mexico", # MICHOACÁN DE OCAMPO
    "17": "Centro de Mexico",   # MORELOS
    "18": "Occidente de Mexico", # NAYARIT
    "19": "Norte de Mexico",    # NUEVO LEÓN
    "20": "Sur de Mexico",      # OAXACA
    "21": "Bajio de Mexico", # PUEBLA
    "22": "Centro de Mexico",   # QUERÉTARO
    "23": "Sur de Mexico",      # QUINTANA ROO
    "24": "Centro de Mexico",   # SAN LUIS POTOSÍ
    "25": "Norte de Mexico",    # SINALOA
    "26": "Norte de Mexico",    # SONORA
    "27": "Sur de Mexico",      # TABASCO
    "28": "Norte de Mexico",    # TAMAULIPAS
    "29": "Bajio de Mexico", # TLAXCALA
    "30": "Sur de Mexico",      # VERACRUZ DE IGNACIO DE LA LLAVE
    "31": "Sur de Mexico",      # YUCATÁN
    "32": "Norte de Mexico" ,    # ZACATECAS
    "36": 'Estados Unidos',
    "97": "No aplica",
    "98": "Se ignora",
    "99": "No especifico"
}

In [14]:
df["ENTIDAD_NAC"]=df["ENTIDAD_NAC"].replace(diccionario_region)
df["ENTIDAD_UM"]=df["ENTIDAD_UM"].replace(diccionario_region)
df["ENTIDAD_RES"]=df["ENTIDAD_RES"].replace(diccionario_region)

In [15]:
df["ENTIDAD_NAC"].value_counts(1)

ENTIDAD_NAC
Centro de Mexico       0.515959
Norte de Mexico        0.196566
Sur de Mexico          0.161968
Occidente de Mexico    0.071842
Bajio de Mexico        0.049388
No especifico          0.004277
Name: proportion, dtype: float64

In [16]:
df["ENTIDAD_UM"].value_counts(1)

ENTIDAD_UM
Centro de Mexico       0.528675
Norte de Mexico        0.217650
Sur de Mexico          0.137826
Occidente de Mexico    0.068423
Bajio de Mexico        0.047425
Name: proportion, dtype: float64

In [17]:
df["ENTIDAD_RES"].value_counts(1)

ENTIDAD_RES
Centro de Mexico       0.525026
Norte de Mexico        0.218011
Sur de Mexico          0.140023
Occidente de Mexico    0.069026
Bajio de Mexico        0.047914
Name: proportion, dtype: float64

In [18]:
categorizar_edad = lambda edad: "mayores a 60" if edad > 60 else f"{(edad // 10) * 10}-{(edad // 10) * 10 + 9}"
df['EDAD'] = df['EDAD'].apply(categorizar_edad)

In [19]:
df['EDAD'].value_counts(1)

EDAD
30-39           0.205497
20-29           0.201568
40-49           0.164238
mayores a 60    0.136159
50-59           0.135570
0-9             0.078125
10-19           0.068646
60-69           0.010198
Name: proportion, dtype: float64

In [20]:
paises = {
    "Africa": [
        "Argelia", "Angola", "Benín", "Botsuana", "Burkina Faso", "Burundi", "Cabo Verde", 
        "Camerún", "Chad", "Comoras", "Costa de Marfil", "Egipto", "Eritrea", "Etiopía", 
        "Gabón", "Gambia", "Ghana", "Guinea", "Guinea-Bisáu", "Guinea Ecuatorial", 
        "Kenia", "Lesoto", "Liberia", "Libia", "Madagascar", "Malaui", "Malí", "Marruecos", 
        "Mauricio", "Mauritania", "Mozambique", "Namibia", "Níger", "Nigeria", 
        "República Centroafricana", "República del Congo", "República Democrática del Congo", 
        "Ruanda", "Santo Tomé y Príncipe", "Senegal", "Seychelles", "Sierra Leona", "Somalia", 
        "Suazilandia (Esuatini)", "Sudáfrica", "Sudán", "Sudán del Sur", "Tanzania", "Togo", 
        "Túnez", "Uganda", "Yibuti", "Zambia", "Zimbabue","República de Angola","Reino de Swazilandia",
        "República Centro Africana","Congo"
    ],
    "America": [
        "Canadá", "Estados Unidos de América", "México", "Argentina", "Bolivia", "Brasil", "Chile", 
        "Colombia", "Ecuador", "Guyana", "Paraguay", "Perú", "Surinam", "Uruguay", "Venezuela","República de Honduras",
        "Guatemala","Nicaragua","Cuba","Haití","El Salvador","Belice","República de Costa Rica",
        "República de Panamá","República Oriental del Uruguay","Jamaica", "República de Trinidad y Tobago",
        "Estado Libre Asociado de Puerto Rico","República Dominicana","República de Guyana"
    ],
    "Asia": [
        "Afganistán", "Arabia Saudita", "Armenia", "Azerbaiyán", "Bangladés", "Baréin", 
        "Birmania (Myanmar)", "Brunéi", "Bután", "Camboya", "Catar", "China", "Chipre", 
        "Corea del Norte", "Corea del Sur", "Emiratos Árabes Unidos", "Filipinas", 
        "Georgia", "India", "Indonesia", "Irak", "Irán", "Israel", "Japón", "Jordania", 
        "Kazajistán", "Kirguistán", "Kuwait", "Laos", "Líbano", "Malasia", "Maldivas", 
        "Mongolia", "Nepal", "Omán", "Pakistán", "Palestina", "Qatar", "Rusia", "Singapur", 
        "Siria", "Sri Lanka", "Tailandia", "Tayikistán", "Timor Oriental", "Turkmenistán", 
        "Turquía", "Uzbekistán", "Vietnam", "Yemen","República de Corea","Macao","Bangladesh",
        "República Democrática de Corea","Reino Hachemita de Jordania","Iraq","Hong Kong"
    ],
    "Europa": [
        "Albania", "Alemania", "Andorra", "Austria", "Bélgica", "Bielorrusia", 
        "Bosnia y Herzegovina", "Bulgaria", "Chipre", "Croacia", "Dinamarca", 
        "Eslovaquia", "Eslovenia", "España", "Estonia", "Finlandia", "Francia", 
        "Grecia", "Hungría", "Irlanda", "Islandia", "Italia", "Letonia", 
        "Liechtenstein", "Lituania", "Luxemburgo", "Macedonia del Norte", 
        "Malta", "Moldavia", "Mónaco", "Montenegro", "Noruega", "Países Bajos", 
        "Polonia", "Portugal", "Reino Unido", "República Checa", "Rumanía", 
        "Rusia (parte europea)", "San Marino", "Serbia", "Suecia", "Suiza", 
        "Ucrania", "Vaticano","Gran Bretaña (Reino Unido)","Holanda","Rumania","Guyana Francesa",
        "Países de la Ex-Yugoslavia","República Checa y República Eslovaca","El Vaticano"
    ],
    "Oceania": [
        "Australia", "Fiyi", "Islas Marshall", "Islas Salomón","Kiribati","Micronesia","Nauru","Nueva Zelanda","Palau",
        "Papúa Nueva Guinea","Samoa","Tonga","Tuvalu","Vanuatu"],
    "NO APLICA": [
        "97"],
    "SE IGNORA": [
        "98","SE DESCONOCE"],
    "NO ESPECIFICADO": [
        "99","Otro","Zona Neutral"]
}
diccio_continente = {continente:  pais for pais, continentes in paises.items() for continente in continentes}

In [21]:
df["PAIS_NACIONALIDAD"]=df["PAIS_NACIONALIDAD"].replace(diccio_continente)
df["PAIS_ORIGEN"]=df["PAIS_ORIGEN"].replace(diccio_continente)

In [22]:
df["PAIS_NACIONALIDAD"].value_counts(1)

PAIS_NACIONALIDAD
America            0.999102
Asia               0.000484
Europa             0.000196
NO ESPECIFICADO    0.000181
Africa             0.000031
Oceania            0.000004
SE IGNORA          0.000003
Name: proportion, dtype: float64

In [23]:
df["PAIS_ORIGEN"].value_counts(1)

PAIS_ORIGEN
NO APLICA          0.998678
America            0.001187
Asia               0.000083
NO ESPECIFICADO    0.000025
Europa             0.000018
Africa             0.000009
Oceania            0.000002
Name: proportion, dtype: float64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137999 entries, 0 to 1137998
Data columns (total 36 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1137999 non-null  object
 1   ID_REGISTRO            1137999 non-null  object
 2   ORIGEN                 1137999 non-null  int64 
 3   SECTOR                 1137999 non-null  int64 
 4   ENTIDAD_UM             1137999 non-null  object
 5   SEXO                   1137999 non-null  int64 
 6   ENTIDAD_NAC            1137999 non-null  object
 7   ENTIDAD_RES            1137999 non-null  object
 8   MUNICIPIO_RES          1137999 non-null  int64 
 9   TIPO_PACIENTE          1137999 non-null  int64 
 10  FECHA_SINTOMAS         1137999 non-null  object
 11  NEUMONIA               1137999 non-null  int64 
 12  EDAD                   1137999 non-null  object
 13  NACIONALIDAD           1137999 non-null  int64 
 14  EMBARAZO               1137999 non

In [25]:
#Considero que es mejor quedarnos con el mes y no con toda la fecha, igual no considero relevante mantener la fecha de actualización
df["FECHA_SINTOMAS"] = pd.to_datetime(df["FECHA_SINTOMAS"])
df["MES_SINTOMAS"] = df["FECHA_SINTOMAS"].dt.month
df=df.drop(columns=["FECHA_ACTUALIZACION","FECHA_SINTOMAS"])

In [26]:
df.shape

(1137999, 35)

In [27]:
df_c=df.copy()

In [ ]:
#Esta parte es para la correlación entre variables
cramersv = am.CramersV(df_c.astype('category')) 
cramersv.fit()

In [25]:
df=df.drop(columns=["ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","PAIS_NACIONALIDAD","RESULTADO_ANTIGENO","INDIGENA"])

In [29]:
df_c=df_c.drop(columns=["ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","PAIS_NACIONALIDAD","RESULTADO_ANTIGENO","INDIGENA"])

In [30]:
X = df_c.drop('TIPO_PACIENTE',axis=1)
y = df_c['TIPO_PACIENTE']

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.7,
                                                    random_state=333,
                                                    shuffle=True,
                                                    stratify = df_c['TIPO_PACIENTE'])

In [31]:
X_train.describe()

,ORIGEN,SECTOR,SEXO,NEUMONIA,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE
count,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,...,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000,796599.000000
mean,1.706994,6.955177,1.411528,2.461822,1.004238,41.537117,6.250106,2.021396,2.080495,2.063546,...,2.076724,2.003895,2.076779,2.052954,3.386437,1.815906,79.497657,1.121961,5.469783,98.665168
std,0.455141,3.698056,0.492111,6.885052,0.064962,46.850508,19.875909,3.116998,2.944252,2.891487,...,2.935053,2.795801,2.905017,3.006466,12.076630,0.387561,36.847640,0.327241,1.930853,5.700446
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000,1.000000,3.000000,99.000000
50%,2.000000,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000,1.000000,7.000000,99.000000
75%,2.000000,12.000000,2.000000,2.000000,1.000000,97.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000,1.000000,7.000000,99.000000
max,2.000000,12.000000,2.000000,99.000000,2.000000,99.000000,99.000000,98.000000,98.000000,98.000000,...,98.000000,98.000000,98.000000,98.000000,99.000000,2.000000,97.000000,2.000000,7.000000,99.000000


In [41]:
preprocesado = ColumnTransformer([
     ('one', OneHotEncoder(sparse=False, drop='first'), df_c)
], remainder='drop')

pipeline = Pipeline([
    ('prepross', preprocesado)
])

X_train_pp = pipeline.fit_transform(X_train, y_train)
X_test_pp = pipeline.transform(X_test)

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed